In [1]:
%cd ../

/Users/macos/Projects/xButler/EvilTransmission/eviltrans-feat_extract


In [2]:
import cv2
import numpy as np
import torch
import onnx
import onnxruntime
import torchvision
from PIL import Image
from torchvision import transforms
from onnxruntime.quantization import quantize_dynamic, QuantType
from onnxconverter_common import float16

In [29]:
def transform_img(img):
    img = img.resize((224, 224), Image.LANCZOS)

    img = np.asarray(img).astype(np.float16)

    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    img = img.transpose([2, 0, 1])
    for channel in range(img.shape[0]):
        img[channel, :, :] = (img[channel, :, :] / 255 - mean[channel]) / std[channel]
    img = np.expand_dims(img, axis=0)

    return img

### Create custom ResNet-34 model

In [20]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)

model.fc = torch.nn.Identity()

model.eval()

Using cache found in /Users/macos/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/macos/Projects/xButler/EvilTransmission/eviltrans-feat_extract/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/macos/Projects/xButler/EvilTransmission/eviltrans-feat_extract/.venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Export ONNX model with bfloat16 enabled

In [21]:
MODEL_NAME = "resnet34_custom.onnx"

x = torch.randn(1, 3, 224, 224, requires_grad=True)
torch_out = model(x)

# Export the model
torch.onnx.export(
    model,               # model being run
    x,                         # model input (or a tuple for multiple inputs)
    MODEL_NAME,
    export_params=True,        # store the trained parameter weights inside the model file
    opset_version=10,          # the ONNX version to export the model to
    do_constant_folding=True,  # whether to execute constant folding for optimization
    input_names = ['input'],   # the model's input names
    output_names = ['output'], # the model's output names
    dynamic_axes={
        'input' : {0 : 'batch_size'},    # variable length axes
        'output' : {0 : 'batch_size'}
    }
)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



#### Convert model to FP16 using native feature of ONNX

In [ ]:
!python -m onnxruntime.quantization.preprocess --input mobilenetv2-7.onnx --output mobilenetv2-7-infer.onnx

In [ ]:
model = onnx.load("resnet34_custom_infer.onnx")
model_fp16 = float16.convert_float_to_float16(model)
onnx.save(model_fp16, "resnet34_custom_infer_quan.onnx")

### Test model

In [40]:
path1 = "/Users/macos/.config/iterm2/naomi_pic/IMG_20210602_002402_401.jpg"
path2 = "/Users/macos/.config/iterm2/naomi_pic/IMG_20210602_002407_936.jpg"

img1, img2 = Image.open(path1), Image.open(path2)

In [41]:
img1_trans, img2_trans = transform_img(img1), transform_img(img2)

In [42]:
ort_session = onnxruntime.InferenceSession("resnet34_custom_infer_quan.onnx")

In [34]:
def get_out(img):
    # compute ONNX Runtime output prediction
    ort_inputs = {ort_session.get_inputs()[0].name: img}
    ort_outs = ort_session.run(None, ort_inputs)

    return ort_outs[0]

In [36]:
def get_sim(x, y):
    def _get_norm(x: np.ndarray, eps: float = 1e-8):
        normed = np.clip(np.linalg.norm(x, axis=-1, keepdims=True), eps, None)

        return normed
    
    x = x / _get_norm(x)
    y = y / _get_norm(y)

    return np.squeeze(x @ y.T)

In [43]:
out1 = get_out(img1_trans)
out2 = get_out(img2_trans)

In [44]:
get_sim(out1, out2)

array(0.7295, dtype=float16)

In [22]:
import onnx
from onnxconverter_common import float16

model = onnx.load("resnet34_custom_infer.onnx")
model_fp16 = float16.convert_float_to_float16(model)
onnx.save(model_fp16, "resnet34_custom_infer_quan.onnx")

/Users/macos/Projects/xButler/EvilTransmission/eviltrans-feat_extract/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 1.0072758048861714e-15 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/Users/macos/Projects/xButler/EvilTransmission/eviltrans-feat_extract/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:53: UserWarning: the float32 number -2.3440090980134023e-15 will be truncated to -1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(neg_max, -min_positive_val))
/Users/macos/Projects/xButler/EvilTransmission/eviltrans-feat_extract/.venv/lib/python3.10/site-packages/onnxconverter_common/float16.py:43: UserWarning: the float32 number 1.298854947207051e-11 will be truncated to 1e-07
  warnings.warn("the float32 number {} will be truncated to {}".format(pos_min, min_positive_val))
/Users/macos/Projects/xButler/Ev